## Sistemas de recomendación

### Análisis exploratorio

In [1]:
##depende del dataset/ observar variables, crear un dataset de items y users

### Preparación del modelo

#### Matrices sparsed

In [ ]:
#creamos la matriz sparsed
import numpy as np
n_users = df.UserID.unique().shape[0]

n_items = df.ItemID.unique().shape[0]

ratings = np.zeros((n_users, n_items))

#transformación a matriz sparsed
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    
#coef. d sparseidad, número de elementos que no tienen ceros (%)
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0]*ratings.shape[1])
sparsity *= 100
print("Coeficiente de sparseidad: {:4.2f}%".format(sparsity))

#### División del dataset

In [ ]:
from sklearn.model_selection import train_test_split
ratings_train, ratings_test = train_test_split(ratings, test_size = 0.3, random_state=42)

### Matriz de similaridad

In [ ]:
import numpy as np
import sklearn

#por ser distancia del coseno hay que transformarla a 1-cosin
sim_matrix = 1 - sklearn.metrics.pairwise.cosine_distances(ratings_train)

users_predictions = sim_matrix.dot(ratings_train) / np.array([np.abs(sim_matrix).sum(axis=1)]).T
#prod matricial sim_matrix*rating_trains/ ponderación por los usuarios de la misma fila

In [ ]:
#función para obtener el MeanSquaredError
from sklearn.metrics import mean_squared_error

def get_mse(preds, actuals):
    if preds.shape[0] != actuals.shape[0]:
        actuals = actuals.T
    preds = preds[actuals.nonzero()].flatten()
    actuals = actuals[actuals.nonzero()].flatten()
    return mean_squared_error(preds, actuals)

get_mse(users_predictions, ratings_train)

In [ ]:
sim_matrix = 1 - sklearn.metrics.pairwise.cosine_distances(ratings_test)
users_predictions = sim_matrix.dot(ratings_test) / np.array([np.abs(sim_matrix).sum(axis=1)]).T
get_mse(users_predictions, ratings_test)

### Filtro colaborativo KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
k=5
neighbors = NearestNeighbors(k, 'cosine')
neighbors.fit(ratings_train)

top_k_distances, top_k_users = neighbors.kneighbors(ratings_train, return_distance=True)

In [ ]:
users_predicts_k = np.zeros(ratings_train.shape)

for i in range(ratings_train.shape[0]):# para cada usuario del conjunto de entrenamiento
    users_predicts_k[i,:] = top_k_distances[i].T.dot(ratings_train[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T

In [ ]:
get_mse(users_predicts_k, ratings_train)

In [ ]:
users_predicts_k = np.zeros(ratings_test.shape)

for i in range(ratings_test.shape[0]):# para cada usuario del conjunto de test

    users_predicts_k[i,:] = top_k_distances[i].T.dot(ratings_test[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T

get_mse(users_predicts_k, ratings_test)

## Filtro colaborativo basado en Items

neighbors = NearestNeighbors(n_movies, 'cosine')
neighbors.fit(ratings_train.T)
top_k_distances, top_k_items = neighbors.kneighbors(ratings_train.T, return_distance=True)

In [ ]:
item_preds = ratings_train.dot(top_k_distances) / np.array([np.abs(top_k_distances).sum(axis=1)])
get_mse(item_preds, ratings_train)

In [ ]:
get_mse(item_preds, ratings_test)

### Filtro colaborativo basado en KNN